## Talent Scheduling Greedy, etwas formalisiert

Sei:
- $k$ der aktuelle Schritt (hier; Anzahl der bereits geordneten Szenen)


- $x_k$ der aktuelle Zustand (hier; die aktuelle Szene und bereits geordnete Szenen)


- $u_k$ eine Entscheidung aus der Menge $U_k(x_k)$ der im Zustand $x_k$ möglichen zulässigen Entscheidungen 
  - hier; eine mögliche noch nicht geordnete Szene
  
  
- $g(x_k, u_k)$ Kosten / Gewinnbeitrag der Entscheidung $u_k$ im Zustand $x_k$
  - hier; die Extra-Kosten, die anfallen, wenn die Szene als nächtes vorkommt/geordnet wird
  
*noch nicht für den Greedy wichtig* 
- $f(x_k, u_k)$ der Folgezustand, der aus $x_k$ und der Entscheidung $u_k$ resultiert (eine Zustandstransition)
  - hier; die erweiterte Menge der besuchten Knoten und der Zielknoten
  

**Bei Greedy** 
- basiert die Entscheidung in der Regel nur auf den (kurzfristigen) Kosten $g(x_k, u_k)$
    $$\min_{u_k \in U_k} g(x_k, u_k)$$

### Implementierung

In [1]:
#IMPORTs
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#numba
from numba import njit, jit
from numba.typed import List

from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

In [2]:
class ts_udp:
    """
    UDP (User-Defined Problem) class for the Talent Scheduling problem of the 
    from the course AuD(Team02[LG,DSc,RK,DSw])

    Summary
    ========
    The talent scheduling problem can be described as follows. A film producer needs 
    to schedule the scenes of his/her movie on a given location. Each scene has a duration (the days it takes to shoot it) 
    and requires some subset of the cast to be on location. The cast are paid for each day they are required to be on 
    location from the day the first scene they are in is shot, to the day the last scene they are in is shot, even though 
    some of those days they might not be required by the scene currently being shot (i.e., they will be on location waiting
    for the next scene they are in to be shot). Each cast member has a different daily salary. The aim of the film producer
    is to order the scenes in such a way as to minimize the salary cost of the shooting. 

    """

    def __init__(self, file_path):
        self.file_path = file_path
        self.name = ""
        self.num_scenes = 0
        self.num_actors = 0
        self.actor_apprs = []
        self.actor_cost = []
        self.scene_duration = []
        self._load_instance()

    def _load_instance(self):
        """
        Load the instance from the dt-file.
        """
        with open(self.file_path, 'r') as file:
            lines = file.readlines()
            self.name = lines[0].strip()  # First line is the name of the data file
            self.num_scenes = int(lines[1].strip())  # Second line is the number of scenes
            self.num_actors = int(lines[2].strip())  # Third line is the number of actors
            
            # Reading actor data
            actor_data = lines[4:4 + self.num_actors]    #entsprechend wird hier angenommen, dass die Instanzen den dasselbe Format besitzen
            for actor_line in actor_data:
                parts = actor_line.strip().split()
                appearance = list(map(int, parts[:-1]))  # convert each part except last to int
                cost = int(parts[-1])  # Last part is the cost
                self.actor_apprs.append(appearance)
                self.actor_cost.append(cost)

            # Reading scene duration
            self.scene_duration = list(map(int, lines[5 + self.num_actors].strip().split()))


    ####################
    ## Public Methods ##
    ####################

    def info(self):
        print(f"File name: {self.name}")
        print(f"Number of scenes: {self.num_scenes}, Number of actors: {self.num_actors}")
        print(f"Actor appearances: {self.actor_apprs}")
        print(f"Actor costs: {self.actor_cost}")
        print(f"Scene durations: {self.scene_duration}")

    def scene_cost(self): #für raw-cost
        # Konvertieren der Liste in ein numpy 2D-Array, für Recheneffizienz
        array_actor_apprs = np.array(self.actor_apprs)
        
        # Array, mit dem multipliziert werden soll
        array_actor_cost = np.array(self.actor_cost)
        
        # Durchführen der Multiplikation mittels Broadcasting für effizienz
        zwischen_ergebnis = array_actor_apprs * array_actor_cost[:, np.newaxis]
        
        array_scene_duration = np.array(self.scene_duration)
        
        # Durchführen der zweiten Multiplikation mittels Broadcasting
        ergebnis = zwischen_ergebnis * array_scene_duration
        
        
        sk=0
        for e in ergebnis:
            sk += e
        #print(f'die Szenen-Kosten {sk}')
        return list(sk)
    
    #######################################Primitive Heuristic###########################################
    def cost_based_sorting_heuristic(self):
        sk = self.scene_cost()
        return [i for i in sorted(range(len(sk)), key=lambda x: sk[x], reverse=False)]
    #####################################################################################################

                

    ### this is for the FINAL SCORE ###

    def reorder_lists(self,order):
        actor_apprs = self.actor_apprs
        scene_durations = self.scene_duration
        
        # Neu ordnen der Szenendauern und Auftritte
        new_scene_durations = [scene_durations[i] for i in order]

        new_actor_apprs = [[actor_apprs[j][i] for i in order] for j in range(len(actor_apprs))]
        
        return new_actor_apprs, new_scene_durations


    def extra_final(self, order):
        actor_apprs, scene_durations = self.reorder_lists(order)
        extra_cost = 0
        for list_index, act_list in enumerate(actor_apprs):
            mod_len = len(act_list)
            while mod_len > 0 and act_list[mod_len - 1] == 0:
                mod_len -= 1

            i = 0
            while i < mod_len - 1:
                if act_list[i] == 1 and act_list[i + 1] == 0:
                    while i + 1 < mod_len and act_list[i + 1] != 1:
                        extra_cost += self.actor_cost[list_index] * scene_durations[i + 1]
                        i += 1
                i += 1
        
        return extra_cost
    


def score(extra):
    sl = film.scene_cost()
    rawcost = sum(sl)
    mincost = rawcost + extra 
    return mincost #+ unnötige Kosten(extra)    

In [3]:
film = ts_udp(file_path = "/mnt/c/Users/Rehma/Desktop/A&D_Project/data_prj/film116.dat")

### Greedy

In [6]:
### SandBox ###
###############

order = [2,1,0]

print(reorder_lists(order))
print(extra(order))

([[0, 1, 1], [0, 0, 0], [1, 1, 1], [0, 0, 0], [0, 0, 1], [1, 0, 0], [1, 1, 0], [0, 0, 1]], [1, 1, 1])
84


In [8]:
%%time
def reorder_lists(order):
    actor_apprs = film.actor_apprs         # werden aus der Instanz in der Klasse eingelesen
    scene_durations = film.scene_duration  # ebenfalls
    
    # Neu ordnen der Szenendauern und Auftritte basierend auf 'order'
    new_scene_durations = [scene_durations[i] for i in order] 
    new_actor_apprs = [[actor_apprs[j][i] for i in order] for j in range(len(actor_apprs))]
    
    return new_actor_apprs, new_scene_durations
    
def extra(order):
        actor_apprs, scene_durations = reorder_lists(order)
        #print(actor_apprs, scene_durations) ################
    
        extra_cost = 0
        for list_index, act_list in enumerate(actor_apprs):
            l = len(act_list) # =19
            i = 0
            while i < l - 1: # wir gehen alle Szenen bis auf die letzte Szene(sonst out of index bei 'i+1') durch
                if act_list[i] == 1 and act_list[i + 1] == 0: # kommt vor 0 eine 1?
                    while i + 1 < l and act_list[i + 1] != 1: # solange wir noch Szenen haben und das nächste Elemente nicht 1 ist; 
                        extra_cost += film.actor_cost[list_index] * scene_durations[i + 1]  # füge die extra-Kosten entsprechend
                        i += 1
                        #print(extra_cost, i,l) #############
                i += 1 # stellt sicher, dass wir über die Szenen(über die Schauspieler) iterieren
        
        return extra_cost

def ts_greedy(start_scene):
    order = [start_scene]   #=tour bei TSP
    remaining_scenes = set(range(film.num_scenes))
    remaining_scenes.remove(start_scene)
    
    while remaining_scenes:
        min_extra_cost = 999999999
        best_scene = None
        
        for scene in remaining_scenes:
            current_order = order + [scene]
            extra_cost = extra(current_order) #extra() kann mit beliebig vielen Szenen in beliebiger Anordnung arbeiten 
            #print(extra_cost, order) ################
            if extra_cost < min_extra_cost: # wenn die Extrakosten der 'current_order' kleiner sind als die bisher bekannten niedrigsten Extrakosten;
                min_extra_cost = extra_cost
                best_scene = scene
        
        order.append(best_scene)
        remaining_scenes.remove(best_scene)

    
    return order

print(ts_greedy(0))
order = ts_greedy(0)
print(f'extra; {film.extra_final(order)}')

[0, 3, 17, 14, 16, 1, 12, 10, 9, 2, 5, 15, 7, 4, 11, 13, 6, 18, 8]
extra; 199
CPU times: user 5.81 ms, sys: 5.87 ms, total: 11.7 ms
Wall time: 11.6 ms


In [ ]:
print(film.actor_apprs)
print('--'*33)
print(film.scene_duration) 
print(film.actor_cost) 
print(film.num_scenes)

### Multi Start Greedy

In [9]:
%%time
def multi_start_ts():
    lowest_extra = 9999999
    best_order = None
    for start_scene in range(len(film.scene_duration)):
        order = ts_greedy(start_scene)
        extra = int(film.extra_final(order))
        if extra < lowest_extra:
            lowest_extra = extra
            best_order = order         

    return lowest_extra, best_order 

multi_start_ts()

CPU times: user 96.7 ms, sys: 0 ns, total: 96.7 ms
Wall time: 96.5 ms


(174, [18, 0, 3, 17, 14, 16, 1, 12, 10, 9, 2, 5, 15, 7, 4, 11, 13, 6, 8])

### Übersicht

In [ ]:
film.info()

print(100 * "~")

print(f'die Szenen-Kosten;     {film.scene_cost()}')

print(100 * "~")

print(f'cost_based_sorting;    {film.cost_based_sorting_heuristic()} extra = {film.extra_final(order = film.cost_based_sorting_heuristic())}')
print(f'ts_greedy;             {ts_greedy(0)} extra = {film.extra_final(order = ts_greedy(0))}')
print(f'multi_start_ts;        {multi_start_ts()[1]} extra = {multi_start_ts()[0]}')

print("die 'optimale' Lösung; [18, 17, 14, 3, 0, 16, 1, 12, 13, 6, 4, 10, 2, 9, 15, 5, 7, 11, 8] extra = 110")

In [ ]:
# die Kosten;
print(f'rowcost; {sum(film.scene_cost())}')
print(f'extra; {film.extra_final(order = film.cost_based_sorting_heuristic())}')
print(f'mincost; {score(film.extra_final(film.cost_based_sorting_heuristic()))} /541')

### Rollout Greedy

In [49]:
%%time
def ts_greedy(start_scene):
    order = start_scene   #=tour bei TSP
    remaining_scenes = set(range(film.num_scenes))
    #print(start_scene)
    if start_scene[0] in remaining_scenes:
        remaining_scenes.remove(start_scene[0])
    
    while remaining_scenes:
        min_extra_cost = 999999999
        best_scene = None
        
        for scene in remaining_scenes:
            current_order = order + [scene]
            extra_cost = extra(current_order) #extra() kann mit beliebig vielen Szenen in beliebiger Anordnung arbeiten 
            #print(extra_cost, order) ################
            if extra_cost < min_extra_cost: # wenn die Extrakosten der 'current_order' kleiner sind als die bisher bekannten niedrigsten Extrakosten;
                min_extra_cost = extra_cost
                best_scene = scene
        
        order.append(best_scene)
        remaining_scenes.remove(best_scene)

    
    return order, film.extra_final(order)
    
def select_using_rollout(start_scene):
    order = [start_scene]
    remaining_scenes = set(range(film.num_scenes))
    #print(remaining_scenes)
    if start_scene in remaining_scenes:
        remaining_scenes.remove(start_scene)
    #print(start_scene)
    node = order[len(order)-1]
    #print(node,len(order)-1)
    best_estimated_value = 1000000
    best_node = node        
        
    for next_scene in remaining_scenes:
        #print(order, next_scene)
        if next_scene in order:
            continue            
        # _, heißt, dass wir den ersten Rückgabewert ignorieren
        
        # Berechne die NN-Länge der Rest-start_scene von next_scene aus
        order.append(next_scene)
        #print(order,next_scene)
        _, nn_value = ts_greedy(order)
        
        
                         # g(x_k, u_k)                    + J~ (f(x_k, u_k))
        estimated_value = extra([node,next_scene]) + nn_value

        if estimated_value < best_estimated_value:
            best_node = next_scene
            best_estimated_value = estimated_value

    
   
    return best_node, extra([node,best_node])

def ts_rollout_greedy(start_scene):
    order = [start_scene]  #:[0]
    remaining_scenes = set(range(film.num_scenes)) #:[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
    remaining_scenes.remove(start_scene) #:[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]
    
    total_extra_cost = 0
    
    #solange die sequenz noch nicht alle Knoten umfasst
    while len(order) < len(remaining_scenes):    #: 1 < 18 -> True
        
        next_scene, extra_cost = select_using_rollout(start_scene) #: 1, 25
        print(next_scene, extra_cost)
        order.append(next_scene)   #: [0,1]
        total_extra_cost += extra_cost     #: 0 + 25 = 25
        
    total_extra_cost += extra([order[len(order)-1],order[0]])  #: 425(=25+25+...) + 4
    print(total_extra_cost,[order[len(order)-1],order[0]] )
    return order, total_extra_cost

ts_rollout_greedy(0)

1 25
1 25
1 25
1 25
1 25
1 25
1 25
1 25
1 25
1 25
1 25
1 25
1 25
1 25
1 25
1 25
1 25
429 [1, 0]
CPU times: user 83.3 ms, sys: 0 ns, total: 83.3 ms
Wall time: 83.1 ms


([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 429)

### Multistart Rollout Greedy

In [ ]:
%%time
def multi_start_rollout():
    lowest_extra = 9999999
    best_order = None
    for start_scene in range(len(film.scene_duration)):
        order = ts_rollout_greedy(start_scene)
        extra = int(film.extra_final(order))
        if extra < lowest_extra:
            lowest_extra = extra
            best_order = order         

    return lowest_extra, best_order 

multi_start_rollout()  

```
tour = order
distance = extra

```

### Breitensuche

In [ ]:
# in progress

### Instanz 'small & small02'

```
========================================== film116
.
.
.

mincost = 541, rawcost = 431, extra = 110, bound = 134546788
ccc = 524287,loop = 4980736
reuse = 4456431, reenter = 0, definite = 0, lookahead = 0, diffuse = 0, clump = 0
FINAL
 18  17  14   3   0  16   1  12  13   6   4  10   2   9  15   5   7  11   8 
----------------------------------------------------------------------------
  .   .   .   X   X   X   X   X   .   .   .   .   .   .   .   .   .   .   . |  10
  .   .   .   .   .   .   .   X   X   -   -   -   -   X   .   .   .   .   . |  4
  .   .   .   .   X   X   X   -   X   X   X   X   X   .   .   .   .   .   . |  5
  .   .   .   .   .   .   .   .   .   .   .   X   -   X   -   -   X   X   X |  5
  X   X   -   -   X   X   -   -   -   -   -   X   -   -   -   X   X   .   . |  5
  .   .   .   .   .   .   .   .   .   .   .   .   X   X   X   X   .   .   . |  40
  .   .   .   .   .   .   X   X   -   -   X   -   X   X   -   -   -   X   . |  4
  .   X   X   X   X   .   .   .   .   .   .   .   .   .   .   .   .   .   . |  20
----------------------------------------------------------------------------
  1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   2 
----------------------------------------------------------------------------
  5  25  25  35  40  20  24  28  18  18  18  23  63  58  54  54  14   9  10  = 541
----------------------------------------------------------------------------
  5  25  20  30  40  20  19  18   9   5   9  15  49  53  40  45  10   9  10 
TIME = 776 ms
FINALDATA 19 8 19 541 524287 4456431 0 776 0 tv film116.dat